# Data
Being able to easily load and process data is a crucial task that can make any data science project more pleasant. In this notebook, we will cover most common types often encountered in data science tasks, and we will be using this data throughout the rest of this tutorial. 

In [18]:
using BenchmarkTools
using DataFrames
using DelimitedFiles
using CSV
using XLSX

# Get some data 
In Julia, it's pretty easy to download a file from the web using the `download` function. But also, you can use your favorite command line command to download files by easily switching from Julia via the `;` key. Let's try both.  
Note `download` depends on external tools such as curl, wget or fetch. So you must have one of these.  

In [2]:
?download

search: download



```
download(url::AbstractString, [path::AbstractString = tempname()]) -> path
```

Download a file from the given url, saving it to the location `path`, or if not specified, a temporary path. Returns the path of the downloaded file.

!!! note
    Since Julia 1.6, this function is deprecated and is just a thin wrapper around `Downloads.download`. In new code, you should use that function directly instead of calling this.



In [3]:
P = download("https://raw.githubusercontent.com/nassarhuda/easy_data/master/programming_languages.csv", 
    "programminglanguages.csv")

"programminglanguages.csv"

Anonther way would be to use a shell command to get the same file.

In [4]:
;wget "https://raw.githubusercontent.com/nassarhuda/easy_data/master/programming_languages.csv"

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\Gow/etc/wgetrc
--2021-03-28 18:06:51--  https://raw.githubusercontent.com/nassarhuda/easy_data/master/programming_languages.csv
Resolving raw.githubusercontent.com... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com|185.199.110.133|:443... connected.
Unable to establish SSL connection.


# Read your data from text files. 
The key question here is to load data from files such as `csv` files, `xlsx` files, or just raw text files. We will go over some Julia packages that will allow us to read such files very easily.  
Let's start with the package `DelimitedFiles` which is in the standard library.

In [5]:
;head programminglanguages.csv

year,language
1951,Regional Assembly Language
1952,Autocode
1954,IPL
1955,FLOW-MATIC
1957,FORTRAN
1957,COMTRAN
1958,LISP
1958,ALGOL 58
1959,FACT


In [11]:
#= 
readdlm(source, 
    delim::AbstractChar, 
    T::Type, 
    eol::AbstarctChar; 
    header=false, 
    skipstart=0,
    skipblanks=treo, 
    use_mmap, 
    quotes=true, 
    dims, 
    comments=false, 
    comment_char='#')
=#
P,H = readdlm("programminglanguages.csv",',';header=true);

In [12]:
H

1×2 Matrix{AbstractString}:
 "year"  "language"

In [13]:
P

73×2 Matrix{Any}:
 1951  "Regional Assembly Language"
 1952  "Autocode"
 1954  "IPL"
 1955  "FLOW-MATIC"
 1957  "FORTRAN"
 1957  "COMTRAN"
 1958  "LISP"
 1958  "ALGOL 58"
 1959  "FACT"
 1959  "COBOL"
 1959  "RPG"
 1962  "APL"
 1962  "Simula"
    ⋮  
 2003  "Scala"
 2005  "F#"
 2006  "PowerShell"
 2007  "Clojure"
 2009  "Go"
 2010  "Rust"
 2011  "Dart"
 2011  "Kotlin"
 2011  "Red"
 2011  "Elixir"
 2012  "Julia"
 2014  "Swift"

In [14]:
# To write to a text file, you can:
writedlm("programminglanguages_dlm.txt", P, '-')

A more powerful package to use here is the `CSV` package. By default, the `CSV` package imports the data to a DataFrame, which can have several advantages as we will see below.  
In general, `CSV.jl` is the recommended way to load CSVs in Julia. Only use `DelimitedFiles` when you have a more complicated file where you want to specify several things.

In [20]:
C = CSV.read("programminglanguages.csv", DataFrame); 

In [24]:
@show typeof(C)
C[1:10,:]
#C[1, :year]

typeof(C) = DataFrame


,year,language
,Int64,String
1,1951,Regional Assembly Language
2,1952,Autocode
3,1954,IPL
4,1955,FLOW-MATIC
5,1957,FORTRAN
6,1957,COMTRAN
7,1958,LISP
8,1958,ALGOL 58
9,1959,FACT


In [25]:
@show typeof(P)
P[1:10,:]

typeof(P) = Matrix{Any}


10×2 Matrix{Any}:
 1951  "Regional Assembly Language"
 1952  "Autocode"
 1954  "IPL"
 1955  "FLOW-MATIC"
 1957  "FORTRAN"
 1957  "COMTRAN"
 1958  "LISP"
 1958  "ALGOL 58"
 1959  "FACT"
 1959  "COBOL"

In [26]:
names(C)

2-element Vector{String}:
 "year"
 "language"

In [27]:
names(C)
C.year
C.language
describe(C)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,year,1982.99,1951,1986.0,2014,0,Int64
2,language,,ALGOL 58,,dBase III,0,String


In [31]:
@btime P,H = readdlm("programminglanguages.csv",',';header=true);
@btime C = CSV.read("programminglanguages.csv", DataFrame);

  173.200 μs (322 allocations: 51.02 KiB)
  221.600 μs (224 allocations: 21.81 KiB)


In [32]:
# To write to a *.csv file using the CSV package
CSV.write("programminglanguages_CSV.csv",DataFrame(P))

"programminglanguages_CSV.csv"

Another type of files that we may often need to read is `XLSX` files. Let's try to read a new file.

In [34]:
T = XLSX.readdata("zillow_data_download_april2020.xlsx", #filename
    "Sale_counts_city", #sheet name
    "A1:F9" #cell range
    )

9×6 Matrix{Any}:
      "RegionID"  "RegionName"    …      "2008-03"      "2008-04"
  6181            "New York"             missing        missing
 12447            "Los Angeles"      1446           1705
 39051            "Houston"          2926           3121
 17426            "Chicago"          2910           3022
  6915            "San Antonio"   …  1479           1529
 13271            "Philadelphia"     1609           1795
 40326            "Phoenix"          1310           1519
 18959            "Las Vegas"        1618           1856

If you don't want to specify cell ranges...though this will take a little longer...

In [35]:
G = XLSX.readtable("zillow_data_download_april2020.xlsx", "Sale_counts_city");

Here, `G` is a tuple of two items. The first is a vector of vectors where each vector corresponds to a column in the excel file. And the second is the header with the column names. 

In [37]:
G[1]

148-element Vector{Any}:
 Any[6181, 12447, 39051, 17426, 6915, 13271, 40326, 18959, 54296, 38128  …  396952, 397236, 398030, 398104, 398357, 398712, 398716, 399081, 737789, 760882]
 Any["New York", "Los Angeles", "Houston", "Chicago", "San Antonio", "Philadelphia", "Phoenix", "Las Vegas", "San Diego", "Dallas"  …  "Barnard Plantation", "Windsor Place", "Stockbridge", "Mattamiscontis", "Chase Stream", "Bowdoin College Grant West", "Summerset", "Long Pond", "Hideout", "Ebeemee"]
 Any["New York", "California", "Texas", "Illinois", "Texas", "Pennsylvania", "Arizona", "Nevada", "California", "Texas"  …  "Maine", "Missouri", "Wisconsin", "Maine", "Maine", "Maine", "South Dakota", "Maine", "Utah", "Maine"]
 Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  28750, 28751, 28752, 28753, 28754, 28755, 28756, 28757, 28758, 28759]
 Any[missing, 1446, 2926, 2910, 1479, 1609, 1310, 1618, 772, 1158  …  0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
 Any[missing, 1705, 3121, 3022, 1529, 1795, 1519, 1856, 1057, 1232  …  0, 0, 0, 0,

In [38]:
G[1][1][1:10]

10-element Vector{Any}:
  6181
 12447
 39051
 17426
  6915
 13271
 40326
 18959
 54296
 38128

In [39]:
G[2][1:10]

10-element Vector{Symbol}:
 :RegionID
 :RegionName
 :StateName
 :SizeRank
 Symbol("2008-03")
 Symbol("2008-04")
 Symbol("2008-05")
 Symbol("2008-06")
 Symbol("2008-07")
 Symbol("2008-08")

and we can easily store this data in a DataFrame. `DataFrame(G...)` uses the "splat" operator to _unwrap_ these arrays and pass them to the DataFrame contructor.

In [41]:
D = DataFrame(G...) # equivalent to DataFrame(G[1], G[2])

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05
,Any,Any,Any,Any,Any,Any,Any
1,6181,New York,New York,1,missing,missing,missing
2,12447,Los Angeles,California,2,1446,1705,1795
3,39051,Houston,Texas,3,2926,3121,3220
4,17426,Chicago,Illinois,4,2910,3022,2937
5,6915,San Antonio,Texas,5,1479,1529,1582
6,13271,Philadelphia,Pennsylvania,6,1609,1795,1709
7,40326,Phoenix,Arizona,7,1310,1519,1654
8,18959,Las Vegas,Nevada,8,1618,1856,1961
9,54296,San Diego,California,9,772,1057,1195


In [57]:
# DataFrames functionality is very friendly.
combine(groupby(D, :StateName),size)

,StateName,x1
,Any,Tuple…
1,New York,"(1332, 148)"
2,California,"(1136, 148)"
3,Texas,"(1589, 148)"
4,Illinois,"(1374, 148)"
5,Pennsylvania,"(2196, 148)"
6,Arizona,"(207, 148)"
7,Nevada,"(90, 148)"
8,Florida,"(672, 148)"
9,Indiana,"(684, 148)"


In [58]:
foods = ["apple", "cucumber", "tomato", "banana"]
calories = [105, 47, 22, 105]
prices = [0.85,1.6, 0.8,0.6]
dataframe_calories = DataFrame(item=foods,calories=calories)
dataframe_prices = DataFrame(item=foods,price=prices)

,item,price
,String,Float64
1,apple,0.85
2,cucumber,1.6
3,tomato,0.8
4,banana,0.6


In [60]:
DF = innerjoin(dataframe_calories,dataframe_prices,on=:item)

,item,calories,price
,String,Int64,Float64
1,apple,105,0.85
2,cucumber,47,1.6
3,tomato,22,0.8
4,banana,105,0.6


In [61]:
# we can also use the DataFrame constructor on a Matrix
DataFrame(T)

,x1,x2,x3,x4,x5,x6
,Any,Any,Any,Any,Any,Any
1,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04
2,6181,New York,New York,1,missing,missing
3,12447,Los Angeles,California,2,1446,1705
4,39051,Houston,Texas,3,2926,3121
5,17426,Chicago,Illinois,4,2910,3022
6,6915,San Antonio,Texas,5,1479,1529
7,13271,Philadelphia,Pennsylvania,6,1609,1795
8,40326,Phoenix,Arizona,7,1310,1519
9,18959,Las Vegas,Nevada,8,1618,1856


You can also easily write data to an XLSX file

In [62]:
# if you already have a dataframe; 
# XLSX.writetable("filename.xlsx", collect(DataFrames.eachcol(df)), DataFrames.names(df))
XLSX.writetable("writefile_using_XLSX2.xlsx",G[1],G[2])

# Importing your data 
Often, the data you want to import is not stored in plain text, and you might want to import different kinds of types. Here we will go over importing `jld`, `npz`, `rda`, and `mat` files. Hopefully, these four will capture the types from four common programming languages used in Data Science (Julia, Python, R, Matlab).  
We will use a toy example here of a very small matrix. But the same syntax will hold for bigger files. 

In [64]:
#using Pkg
#Pkg.add("JLD")
using JLD

    Updating registry at `C:\Users\malon\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
   Installed Blosc_jll ─── v1.14.3+1
   Installed HDF5_jll ──── v1.12.0+1
   Installed OpenSSL_jll ─ v1.1.1+6
   Installed Lz4_jll ───── v1.9.2+2
   Installed Blosc ─────── v0.7.0
   Installed Requires ──── v1.1.3
   Installed JLD ───────── v0.12.2
   Installed HDF5 ──────── v0.15.4
   Installed Zstd_jll ──── v1.4.8+0
   Installed FileIO ────── v1.6.5
    Updating `C:\Users\malon\.julia\environments\v1.6\Project.toml`
  [4138dd39] + JLD v0.12.2
    Updating `C:\Users\malon\.julia\environments\v1.6\Manifest.toml`
  [a74b3585] + Blosc v0.7.0
  [5789e2e9] + FileIO v1.6.5
  [f67ccb44] + HDF5 v0.15.4
  [4138dd39] + JLD v0.12.2
  [ae029012] + Requires v1.1.3
  [0b7ba130] + Blosc_jll v1.14.3+1
  [0234f1f7] + HDF5_jll v1.12.0+1
  [5ced341a] + Lz4_jll v1.9.2+2
  [458c3c95] + OpenSSL_jll v1.1.1+6
  [3161d3a3] + Zstd_jll v1.4.8

In [65]:
jld_data = JLD.load("mytempdata.jld")
save("mywrite.jld", "A", jld_data)

Dict{String, Any} with 1 entry:
  "tempdata" => [2 1446 … 1795 1890; 3 2926 … 3220 3405; 4 2910 … 2937 3224; 5 …

In [66]:
#Pkg.add("NPZ")
using NPZ
npz_data = npzread("mytempdata.npz")
npzwrite("mywrite.npz", npz_data)

   Resolving package versions...
   Installed NPZ ─ v0.4.1
    Updating `C:\Users\malon\.julia\environments\v1.6\Project.toml`
  [15e1cf62] + NPZ v0.4.1
    Updating `C:\Users\malon\.julia\environments\v1.6\Manifest.toml`
  [15e1cf62] + NPZ v0.4.1
Precompiling project...
  ✓ NPZ
1 dependency successfully precompiled in 3 seconds (51 already precompiled)


In [67]:
#Pkg.add("RData")
using RData

   Resolving package versions...
   Installed RData ─────── v0.8.0
   Installed ExprTools ─── v0.1.3
   Installed RecipesBase ─ v1.1.1
   Installed TimeZones ─── v1.5.3
    Updating `C:\Users\malon\.julia\environments\v1.6\Project.toml`
  [df47a6cb] + RData v0.8.0
    Updating `C:\Users\malon\.julia\environments\v1.6\Manifest.toml`
  [944b1d66] + CodecZlib v0.7.0
  [e2ba6199] + ExprTools v0.1.3
  [78c3b35d] + Mocking v0.7.1
  [df47a6cb] + RData v0.8.0
  [3cdcf5f2] + RecipesBase v1.1.1
  [f269a46b] + TimeZones v1.5.3
  [3bb67fe8] + TranscodingStreams v0.9.5
    Building TimeZones → `C:\Users\malon\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\4ba8a9579a243400db412b50300cd61d7447e583\build.log`
Precompiling project...
  ✓ TranscodingStreams
  ✓ CodecZlib
  ✓ RData
3 dependencies successfully precompiled in 7 seconds (56 already precompiled)


In [78]:
#ENV["R_HOME"] = ""
ENV["R_HOME"] = "C:\\Program Files\\R\\R-4.0.2"
Pkg.build("RCall")
using RCall

    Building Conda → `C:\Users\malon\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\6231e40619c15148bcb80aa19d731e629877d762\build.log`
    Building RCall → `C:\Users\malon\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\9b85a9fd61d9ad3986e35da8b6fa7dc6d2354b75\build.log`
┌ Info: Precompiling RCall [6f49c342-dc21-5d91-9882-a32aef131414]
└ @ Base loading.jl:1317


In [79]:
R_data = RData.load("mytempdata.rda")
# We'll need RCall to save here. https://github.com/JuliaData/RData.jl/issue/36
@rput R_data
R"save(R_data, file=\"mywrite.rda\")"

RObject{NilSxp}
NULL


In [80]:
#Pkg.add("MAT")
using MAT

   Resolving package versions...
   Installed MAT ─ v0.10.1
    Updating `C:\Users\malon\.julia\environments\v1.6\Project.toml`
  [23992714] + MAT v0.10.1
    Updating `C:\Users\malon\.julia\environments\v1.6\Manifest.toml`
  [e1450e63] + BufferedStreams v1.0.0
  [23992714] + MAT v0.10.1
Precompiling project...
  ✓ BufferedStreams
  ✓ MAT
2 dependencies successfully precompiled in 7 seconds (70 already precompiled, 1 skipped during auto due to previous errors)


In [81]:
Matlab_data = matread("mytempdata.mat")
matwrite("mywrie.mat",Matlab_data)

In [83]:
@show typeof(jld_data)
@show typeof(npz_data)
@show typeof(R_data)
@show typeof(Matlab_data)
;

typeof(jld_data) = Dict{String, Any}
typeof(npz_data) = Matrix{Int64}
typeof(R_data) = Dict{String, Any}
typeof(Matlab_data) = Dict{String, Any}


In [88]:
jld_data["tempdata"]

4×5 Matrix{Int64}:
 2  1446  1705  1795  1890
 3  2926  3121  3220  3405
 4  2910  3022  2937  3224
 5  1479  1529  1582  1761

# Time to process the data from Julia
We will mainly cover `Matrix` (or `Vector`), `DataFrame`s, and `dict`s (or dictionaries). Let's bring back our programming languages dataset and start playing it the matrix it's stored in. 

In [89]:
P

73×2 Matrix{Any}:
 1951  "Regional Assembly Language"
 1952  "Autocode"
 1954  "IPL"
 1955  "FLOW-MATIC"
 1957  "FORTRAN"
 1957  "COMTRAN"
 1958  "LISP"
 1958  "ALGOL 58"
 1959  "FACT"
 1959  "COBOL"
 1959  "RPG"
 1962  "APL"
 1962  "Simula"
    ⋮  
 2003  "Scala"
 2005  "F#"
 2006  "PowerShell"
 2007  "Clojure"
 2009  "Go"
 2010  "Rust"
 2011  "Dart"
 2011  "Kotlin"
 2011  "Red"
 2011  "Elixir"
 2012  "Julia"
 2014  "Swift"

Here are some quick questions we might want to ask about this simple data.
* Which year was a given language invented?
* How many languages were created in a given year?

In [93]:
# Q1: Which year was a given language invented? 
function year_created(P, language::String)
    loc = findfirst(P[:,2] .== language)
    return P[loc,1]
end
year_created(P, "Julia")

2012

In [94]:
year_created(P,"W")

LoadError: ArgumentError: invalid index: nothing of type Nothing

In [97]:
function year_created_handle_error(P,language::String)
    loc = findfirst(P[:,2] .== language)
    !isnothing(loc) && return P[loc,1]
    error("Error: language not found.")
end
year_created_handle_error(P,"W")

LoadError: Error: language not found.

In [98]:
# Q2: How many languages were created in a given year?
function how_many_per_year(P,year::Int64)
    year_count = length(findall(P[:,1].==year))
    return year_count
end
how_many_per_year(P,2011)

4

Now let's try to store this data in a DataFrame...

In [99]:
P_df = C #DataFrame(year = P[:,1], language = P[:2]) # or DataFrame(P)

,year,language
,Int64,String
1,1951,Regional Assembly Language
2,1952,Autocode
3,1954,IPL
4,1955,FLOW-MATIC
5,1957,FORTRAN
6,1957,COMTRAN
7,1958,LISP
8,1958,ALGOL 58
9,1959,FACT


In [100]:
# Even better, since we know the types of each column, we can create the DataFrame as follows:
# P_df = DataFrame(year = Int.(P[:,1]), language = string.(P[:,2]))

And now let's answer the same questions we just answered...

In [101]:
# Q1: Which year was a given language invented?
# it's a little more intuitive and you don't need to remember the column ids
function year_created(P_df,language::String)
    loc = findfirst(P_df.language.==language)
    return P_df.year[loc]
end
year_created(P_df,"Julia")

2012

In [102]:
year_created(P_df,"W")

LoadError: ArgumentError: invalid index: nothing of type Nothing

In [104]:
function year_created_handle_error(P_df,language::String)
    loc = findfirst(P_df.language.==language)
    !isnothing(loc) && return P_df.year[loc]
    error("Error: language not found.")
end
year_created_handle_error(P_df,"W")

LoadError: Error: language not found.

In [105]:
# Q2: How many languages were created in a given year? 
function how_many_per_year(P_df,year::Int64)
    year_count = length(findall(P_df.year.==year))
    return year_count
end
how_many_per_year(P_df, 2011)

4

Next, we'll use dictionaries. A quick way to create a dictionary is with the `Dict()` command. But this creates a dictionary without types. Here, we will specify the types of this dictionary.

In [109]:
# A quick example to show how to build a dictioray
Dict([("A",1), ("B",2), ("j", [1,2])])

Dict{String, Any} with 3 entries:
  "B" => 2
  "A" => 1
  "j" => [1, 2]

In [114]:
P_dictionary = Dict{Integer, Vector{String}}()

Dict{Integer, Vector{String}}()

In [115]:
P_dictionary[67] = ["julia", "programming"]

2-element Vector{String}:
 "julia"
 "programming"

In [116]:
# This is not gonna work. 
P_dictionary["julia"] = 7

LoadError: MethodError: [0mCannot `convert` an object of type [92mString[39m[0m to an object of type [91mInteger[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::Base.TwicePrecision[39m) where T<:Number at twiceprecision.jl:250
[0m  convert(::Type{T}, [91m::AbstractChar[39m) where T<:Number at char.jl:180
[0m  convert(::Type{T}, [91m::CartesianIndex{1}[39m) where T<:Number at multidimensional.jl:136
[0m  ...

Now, let's populate the dictionary with years as keys and vectors that hold all the programming languages created in each year as their values. Even though this looks like more work, we often need to do it just once.

In [118]:
dict = Dict{Integer,Vector{String}}()
for i = 1:size(P,1)
    year,lang = P[i,:]
    if year in keys(dict)
        dict[year] = push!(dict[year],lang)
        # note that push! is not our favorite thing to do in Julia,
        # but we're focusing on correctness rather than speed here
    else 
        dict[year] = [lang]
    end
end

In [121]:
dict[2011]

4-element Vector{String}:
 "Dart"
 "Kotlin"
 "Red"
 "Elixir"

In [123]:
# Though a smarter way to do this is:
curyear = P_df.year[1]
P_dictionary[curyear] = [P_df.language[1]]
for (i,nextyear) in enumerate(P_df.year[2:end])
    if nextyear == curyear
        # same key
        P_dictionary[curyear] = push!(P_dictionary[curyear],P_df.language[i+1])
        # note that push! is not our favorite thing to do in Julia, 
        # but we're focusing on correctness rather than speed here
    else
        curyear = nextyear
        P_dictionary[curyear] = [P_df.language[i+1]]
    end
end

In [124]:
length(keys(P_dictionary))

46

In [128]:
# Q1: Which year was a given language invented?
# now instead of looking in one long vector, we will look in many small vectors
function year_created(P_dictionary,language::String)
    keys_vec = collect(keys(P_dictionary))
    lookup = map(keyid -> findfirst(P_dictionary[keyid].==language),keys_vec)
    # now the lookup vector has `nothing` or a numeric value. We want to find the index of the numeric value.
    return keys_vec[findfirst((!isnothing).(lookup))]
end
year_created(P_dictionary,"Julia")

2012

In [126]:
# Q2: How many languages were created in a given year?
how_many_per_year(P_dictionary,year::Int64) = length(P_dictionary[year])
how_many_per_year(P_dictionary,2011)

4

# A note about missing data

In [130]:
# assume there were missing values i nour dataframe
P[1,1] = missing
P_df = DataFrame(year = P[:,1], language = P[:,2])

,year,language
,Any,Any
1,missing,Regional Assembly Language
2,1952,Autocode
3,1954,IPL
4,1955,FLOW-MATIC
5,1957,FORTRAN
6,1957,COMTRAN
7,1958,LISP
8,1958,ALGOL 58
9,1959,FACT


In [131]:
dropmissing(P_df)

,year,language
,Any,Any
1,1952,Autocode
2,1954,IPL
3,1955,FLOW-MATIC
4,1957,FORTRAN
5,1957,COMTRAN
6,1958,LISP
7,1958,ALGOL 58
8,1959,FACT
9,1959,COBOL


# Finally...
After finishing this notebook, you should be able to:
* dowload a data file from the web given a url
* load data from a file from a text file via `DelimitedFiles` or `CSV`
* write your data to a text file or csv file 
* load data from file types xlsx, jld, npz, mat, rda
* write your data to an xlsx file, jld, npz, mat, rda
* store data in a 2D array (`Matrix`), or `DataFrame` or `Dict`
* write functions to perform basic lookups on `Matrix`, `DataFrame`, and `Dict` types
* use some of the basic functions on `DataFrame`s such as: `dropmissing`, `describe`, `groupby`, and `innerjoin`

# One cool finding
Julia was created in 2012